In [1]:
from two_stream_loader import *
import skimage
import pickle
from torch.nn.functional import softmax
import matplotlib.pyplot as plt

from scipy.special import softmax as scipy_softmax

In [2]:
# path_temporal = r"/mnt/disks/datastorage/videos/elp/demos/flownet2/"
# path_spatial = r"/mnt/disks/datastorage/videos/elp/demos/rgb/"

path_temporal = r"/mnt/disks/datastorage/videos/flownet2/"
path_spatial = r"/mnt/disks/datastorage/videos/rgb/"

# path_temporal = r"/mnt/disks/datastorage/videos/elp/test/flownet2/"
# path_spatial = r"/mnt/disks/datastorage/videos/elp/test/rgb/"

# light demos
#file = "v_Assault_g17_c000004"
#file = "v_Shooting_g18_c000004"
#file = "v_Theft_g16_c000004"
#file = "v_Misc_g16_c000006" # works when adding together

# Dark demos
#file = "v_Assault_g12_c000004"
#file = "v_Shooting_g18_c000005"
#file = "v_Theft_g23_c000016"
#file = "v_Misc_g17_c000003"

#file = "v_Shooting_g11_c000001"
#file = "v_Misc_g29_c000005"
file = "v_Shooting_g18_c000001"


imgs_u,imgs_v = load_temporal_images(path_temporal,file)
imgs_spatial = load_spatial_images(path_spatial,file)

imgs_normal = load_normal_spatial_images(path_spatial,file)
imgs_u_normal,imgs_v_normal = load_normal_temporal_images(path_temporal,file)

In [3]:
nb_classes = 4;
[temporal_net,spatial_net] =load_models(spatial_path=r"/home/mlp/two-stream-action-recognition/record/spatial/model_best_FENS.pth.tar",
                temporal_path=r"/home/mlp/two-stream-action-recognition/record/motion/model_best_FENS.pth.tar",
                                        nb_classes=nb_classes,p_optical=.1)

4
Dropout is  0.1
Dropout is  0.1


In [9]:
def stack_op(imgs_u,imgs_v,i):
    _, img_rows, img_cols = imgs_u[0].shape
    flow = torch.FloatTensor(2*10,img_rows,img_cols)

    for j in range(i, 10+i):
        H = imgs_u[j]
        V = imgs_v[j]

        flow[2*(j-1-i),:,:] = H
        flow[2*(j-1-i)+1,:,:] = V

    flow = flow.unsqueeze(0)
    
    return flow

In [ ]:
if 0:
    spatial_net.eval()
    for i in imgs_spatial:
        pred=spatial_net.forward(i.unsqueeze(0)).detach().numpy().squeeze()
        print(spatial_net.forward(i.unsqueeze(0)).detach().numpy().squeeze()," Predict ",pred.argmax())

In [ ]:
if 0:
    temporal_net.eval()
    for i in range(len(imgs_u)-10):
        flow = stack_op(imgs_u,imgs_v,i)
        pred = temporal_net.forward(flow).detach().numpy().squeeze()
        print(pred," Predict ",pred.argmax())
    #     print(temporal_net.forward(flow).argmax(1))

In [4]:
# Load from file
model_pickle_name = "/home/mlp/two-stream-action-recognition/fens_svm.pkl"
with open(model_pickle_name, 'rb') as file:
    svm_model = pickle.load(file)

In [ ]:
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")

    return cv2.LUT(image, table)

In [ ]:
def adjust_gamma(image, gamma=1.0):

    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

    return cv2.LUT(image, table)

In [ ]:
%matplotlib inline 

In [5]:
video_dir = '/mnt/disks/datastorage/videos/'
#file = 'v_Shooting_g11_v1_b.mov'
file = 'v_Shooting_g18_v1_c.MP4'
video_path = os.path.join(video_dir,'fens',file)

In [6]:
video_dir = os.path.dirname(os.path.abspath(video_path))
video_name = os.path.splitext(os.path.basename(video_path))[0]
pickle_path = os.path.join(video_dir, '{}_predictions.pkl'.format(video_name))
use_svm = True

with open(pickle_path, 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    predictions_temporal = scipy_softmax(predictions[:, :nb_classes],1)
    predictions_spatial = scipy_softmax(predictions[:, nb_classes:(nb_classes*2)],1)
    pred_features = np.hstack((predictions_spatial,predictions_temporal))
    predictions_combined = predictions[:, (nb_classes*2):]
    if use_svm:
        actions_prob = svm_model.predict_proba(pred_features)
        actions = svm_model.predict(pred_features)-1
    else:
        actions = np.argmax(predictions_combined, axis=1) 

In [ ]:
from IPython.display import clear_output

temporal_net.eval()
spatial_net.eval()

action_dict = {
    0: 'Theft',
    1: 'Assault',
    2: 'Shooting',
    3: 'No Action',
}
# action_dict = {
#     0: 'Assault',
#     1: 'Shooting',
#     2: 'No Action',
# }

out = cv2.VideoWriter("demo.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20.0, (1344,896), True)
for i in range(len(imgs_spatial)-10):
    rgb = imgs_spatial[i]
    spatial_preds = softmax(spatial_net.forward(rgb.unsqueeze(0).cuda()),1)

    flow = stack_op(imgs_u, imgs_v, i)
    temporal_preds = softmax(temporal_net.forward(flow.cuda()),1)
    #preds = torch.mean(torch.cat((spatial_preds,temporal_preds)),0)#torch.mean(spatial_preds + temporal_preds,0)
    #preds = spatial_preds.squeeze()
    #p = (spatial_preds + temporal_preds).cpu().detach().numpy().squeeze()/2

    p = np.hstack((spatial_preds.cpu().detach().numpy().squeeze(),temporal_preds.cpu().detach().numpy().squeeze()))
    p = svm_model.predict_proba(np.expand_dims(p,0))
    
    label = action_dict[p.argmax().item()]
    
    rgb = imgs_normal[i].copy()
    #rgb = adjust_gamma(imgs_normal[i].copy(),.3)
    
    flow_u = skimage.color.gray2rgb(imgs_u_normal[i].copy())
    flow_v = skimage.color.gray2rgb(imgs_v_normal[i].copy())
    
    # Linearly scale to make pretty!
    flow_u = (flow_u-flow_u.min())/(flow_u.max()-flow_u.min())
    flow_v = (flow_v-flow_v.min())/(flow_v.max()-flow_v.min())
    
    plt.bar(np.arange(0,nb_classes),spatial_preds.cpu().detach().numpy().reshape((-1,)),tick_label=list(action_dict.values()))
    plt.title("Spatial")
    plt.ylim([0,1])
    plt.savefig("SpatialBar.jpg")
    spatial_bar = plt.imread("SpatialBar.jpg")
    spatial_bar = cv2.resize(spatial_bar,(224,224))/255
    plt.close()
    
    plt.bar(np.arange(0,nb_classes),temporal_preds.cpu().detach().numpy().reshape((-1,)),tick_label=list(action_dict.values()))
    plt.title("Temporal")
    plt.ylim([0,1])
    plt.savefig("TemporalBar.jpg")
    temporal_bar = plt.imread("TemporalBar.jpg")
    temporal_bar = cv2.resize(temporal_bar,(224,224))/255
    plt.close()
    
    plt.bar(np.arange(0,nb_classes),p.squeeze(),tick_label=list(action_dict.values()))
    plt.title("Combined")
    plt.ylim([0,1])
    plt.savefig("CombinedBar.jpg")
    combined_bar = plt.imread("CombinedBar.jpg")
    combined_bar = cv2.resize(combined_bar,(224,224))/255
    plt.close()
    
    frame1 = np.hstack((rgb/255, flow_u, flow_v))
    frame2 = np.hstack((combined_bar,spatial_bar,temporal_bar))
    frame = np.vstack((frame1,frame2))
    
    cv2.putText(
        frame,
        label,
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.75,
        (1, 1, 1),
        thickness=2
    )
    frame = cv2.resize(frame, (0, 0), fx=2, fy=2)
    frame = np.uint8(frame * 255)
    out.write(frame)
    
#     plt.figure(1,figsize = (30,30)) # creates a figure 10 inches by 10 inches
#     plt.axis('off')
#     plt.imshow(frame)
#     plt.show()
#     clear_output(wait=True)
#     if i < 5:
#         continue
#     if i > 15:
#         break
#     print(i)

out.release()

In [10]:
from IPython.display import clear_output

temporal_net.eval()
spatial_net.eval()

action_dict = {
    0: 'Theft',
    1: 'Assault',
    2: 'Shooting',
    3: 'No Action',
}
# action_dict = {
#     0: 'Assault',
#     1: 'Shooting',
#     2: 'No Action',
# }
labels = []
temporal_pred = []
spatial_pred = []
for i in range(len(imgs_spatial)-10):
    rgb = imgs_spatial[i]
    spatial_preds = softmax(spatial_net.forward(rgb.unsqueeze(0).cuda()),1)
    spatial_pred.append(spatial_preds.detach().cpu().numpy())

    flow = stack_op(imgs_u, imgs_v, i)
    temporal_preds = softmax(temporal_net.forward(flow.cuda()),1)
    temporal_pred.append(temporal_preds.cpu().detach().numpy())
    
    #preds = torch.mean(torch.cat((spatial_preds,temporal_preds)),0)#torch.mean(spatial_preds + temporal_preds,0)
    #preds = spatial_preds.squeeze()
    #p = (spatial_preds + temporal_preds).cpu().detach().numpy().squeeze()/2

    p = np.hstack((spatial_preds.cpu().detach().numpy().squeeze(),temporal_preds.cpu().detach().numpy().squeeze()))
    p = svm_model.predict_proba(np.expand_dims(p,0))
    
    label = action_dict[p.argmax().item()]
    labels.append(p.argmax().item())

In [13]:
spatial_pred = np.concatenate(spatial_pred)

array([[8.73169396e-03, 8.96788463e-02, 8.89425337e-01, 1.21641224e-02],
       [8.23722593e-03, 8.81016552e-02, 8.93618166e-01, 1.00430222e-02],
       [8.14862456e-03, 6.18803240e-02, 9.16702509e-01, 1.32686142e-02],
       [9.24860965e-03, 6.12379462e-02, 9.18237567e-01, 1.12758465e-02],
       [1.04142036e-02, 4.94338498e-02, 9.30993795e-01, 9.15809441e-03],
       [1.05711697e-02, 4.45067436e-02, 9.33082223e-01, 1.18398592e-02],
       [1.20577756e-02, 5.07276580e-02, 9.24511552e-01, 1.27031133e-02],
       [1.46754235e-02, 5.27351871e-02, 9.19232488e-01, 1.33567872e-02],
       [6.55597076e-03, 2.92919148e-02, 9.58380342e-01, 5.77178271e-03],
       [1.04101356e-02, 7.37543851e-02, 9.07918930e-01, 7.91657344e-03],
       [9.76230297e-03, 1.25151038e-01, 8.46217215e-01, 1.88694652e-02],
       [1.14420503e-02, 8.38079825e-02, 8.89186740e-01, 1.55632636e-02],
       [7.87639711e-03, 1.16931096e-01, 8.59569192e-01, 1.56232752e-02],
       [9.73071344e-03, 9.79827270e-02, 8.79344523e

In [14]:
temporal_pred = np.concatenate(temporal_pred)

In [34]:
spatial_pred = np.concatenate(spatial_pred)

In [22]:
predictions_temporal[0:len(temporal_pred),:].shape

(127, 4)

In [32]:
predictions_spatial[0:len(spatial_pred),:].shape

(127, 4)

In [35]:
(spatial_pred-predictions_spatial[0:len(spatial_pred),:]).max(1)

array([0.87875795, 0.82360154, 0.8366462 , 0.8508769 , 0.8764015 ,
       0.868197  , 0.8215692 , 0.77737683, 0.8145846 , 0.697953  ,
       0.68301046, 0.8340052 , 0.60298985, 0.6570479 , 0.5435574 ,
       0.5672279 , 0.73534113, 0.5176944 , 0.6212846 , 0.6501465 ,
       0.44038996, 0.297898  , 0.5884671 , 0.28728694, 0.07253838,
       0.02153254, 0.07115871, 0.03250262, 0.0559746 , 0.02058405,
       0.02433264, 0.22665638, 0.6600785 , 0.7822607 , 0.5784272 ,
       0.45183057, 0.34637588, 0.02207565, 0.00468462, 0.00588297,
       0.00474253, 0.00436881, 0.00455042, 0.00438447, 0.00404035,
       0.00418743, 0.00762296, 0.00635469, 0.00712022, 0.00734178,
       0.00911629, 0.04062921, 0.03252423, 0.02524281, 0.01510155,
       0.14612979, 0.29919368, 0.10856044, 0.10822278, 0.17057109,
       0.19985521, 0.5418469 , 0.9572179 , 0.9422041 , 0.93501425,
       0.5483049 , 0.07512558, 0.08455253, 0.03198022, 0.00798689,
       0.00541207, 0.01988494, 0.00494676, 0.00846725, 0.00852

In [28]:
(temporal_pred-predictions_temporal[0:len(temporal_pred),:]).max(1)

(127, 4)

In [ ]:
labels = np.array(labels)

In [ ]:
action_labels = actions[0:len(labels)]

In [ ]:
np.sum(action_labels == labels)/len(action_labels)